In [1]:
from threading import Thread
from time import sleep
import socket
import pickle
from Crypto.PublicKey import RSA
from Crypto import Random
from Crypto.Cipher import AES
from Crypto.Signature.pkcs1_15 import PKCS115_SigScheme
from Crypto.Cipher import PKCS1_v1_5 as Cipher_PKCS1_v1_5
from Crypto.Hash import SHA256

import os
import hashlib

HOST = "192.168.43.105"  # Standard loopback interface address (localhost)
PORT = 9999  # Port to listen on (non-privileged ports are > 1023)

In [2]:
def generate_RSA_key():
    keys = RSA.generate(2048)
    pub  = RSA.import_key(keys.public_key().export_key('DER'))
    prv  = RSA.import_key(keys.export_key('DER'))
    return prv,pub

def load_RSA_key():
    if(os.path.exists("userkey.pickle")):
        try:
            f=open("userkey.pickle","rb")
            prv,pub = pickle.load(f)
            prv = RSA.import_key(prv)
            pub = RSA.import_key(pub)
            return prv,pub
        except Exception as e:
            print(e)
            return None
    else:
        return None

def save_RSA_key(prv,pub):
    keyfile = open("userkey.pickle","wb")
    pickle.dump([prv.export_key(),pub.export_key()],keyfile)
    keyfile.close()
    print("Write success")
    return True

def encrypt(key, data):
    enc_data = Cipher_PKCS1_v1_5.new(key).encrypt(data)
    return enc_data

def decrypt(key,enc_data):
    '''
    Params
        key - Key for decryption of message
        enc - Encrypted message
    '''
    data = Cipher_PKCS1_v1_5.new(key).decrypt(enc_data,None)
    return data

def sign(keypair,data):
    digest = SHA256.new(data)
    signer = PKCS115_SigScheme(keypair)
    signature = signer.sign(digest)
    return signature


def verify(enc_message,signature,publickey):
    '''
    Params
        enc_message - Encrypted Message to be verified.
        signature - Signature of the message.
        publickey - Public of the Private key used to create signature
    '''
    data,enc_og,pubkey = enc_message,signature,publickey
    digest = SHA256.new(data)
    verifier = PKCS115_SigScheme(publickey)
    try:
        verifier.verify(digest, signature)
        return True
    except:
        return False

def insertDB(dec_message,public_key,torurl):
    global db
    if(torurl not in db):
        db[torurl]={"messages":[],"pubkey":public_key}
    db[torurl]["messages"].append(dec_message)
    
def process_message(data):
    # Format [torurl,enc_message,signature,public_key]
    data = pickle.loads(data)
    torurl,enc_message,signature,public_key = data
    public_key = RSA.import_key(public_key)
    if(verify(enc_message,signature,public_key)):
        dec_message = decrypt(prv,enc_message)
        insertDB(dec_message,public_key,torurl)
        return True
    else:
        print("Verification Failure")
        return False
# Python Server
def receiver_function():
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.bind(("0.0.0.0", PORT))
        s.listen()
        while True:
            conn, addr = s.accept()
            with conn:
                # TODO: Threading might give better performance
                while True:
                    # Assuming 1024 bytes as max message size
                    data = conn.recv(1024)
                    if not data:
                        break
                    process_message(data)
    except KeyboardInterrupt:
        print("Bye..")
        

In [3]:
db = {}
keys = load_RSA_key()
prv,pub = (None,None)
if(keys==None):
    print("Keys not avaiable, generating")
    prv,pub = generate_RSA_key()
    print("Saving keys")
    save_RSA_key(prv,pub)
else:
    prv,pub = keys
thread_rec = Thread(target=receiver_function)

In [4]:
thread_rec.start()

In [11]:
def client_tester():
    HOST = "localhost"
    PORT = 9999
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.connect((HOST, PORT))
        keys = generate_RSA_key() # Priv,Pub
        message = "Hello MOON!"
        enc_message = encrypt(pub,message.encode())
        signature = sign(keys[0],enc_message)
        s.send(pickle.dumps(["localhost",enc_message,signature,keys[1].export_key()]))
        s.close()

In [12]:
client_tester()

In [13]:
db

{'localhost': {'messages': [b'Hello World!', b'Hello World!', b'Hello MOON!'],
  'pubkey': RsaKey(n=20728589697939446218122343092697448964271927518695104649021930819086106982135777329442158491010227439835488420911159390780219053861158469952987695800527285122228443124293296165496459218704485015215369423064336756796737700566097113084191185893370635294377488379831330895358997571902935994681178736321847294678021826283951577500893751164435101399478428003188276308484278938849840262029637558459804800483980235064838114518187172065282929546199429812671433359542884653071731372149942054903810638286312828452745761593036279544211572081930381082512114917973143570027879441412444773163395810024652514659863966791051236895627, e=65537)}}